In [1]:
import torch
import timm
import sys
sys.path.append('../')
from tokenrank_vit import TokenRankVisionTransformer
from timm.models.vision_transformer import VisionTransformer
from fvcore.nn import FlopCountAnalysis, flop_count_table, flop_count_str
from beit import BeitTeacher
from tokenrank_vit import TokenRankVisionTransformer
from tokenrank_beit import TokenRankBeit
from PIL import Image
from transformers import ViTFeatureExtractor
import requests
%load_ext autoreload
%autoreload 2

In [2]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
feature_extractor = ViTFeatureExtractor()
inputs = feature_extractor(images=image, return_tensors="pt")['pixel_values']

In [10]:
#Testing DEIT -S
patch_size = 16
layers = 12
#prune_list = [2,3,4,5,6,7,8,9,10]
prune_list = [2,5,8]
heads = 6
mlp_ratio = 4.
dims = 384
qkv_bias = True
#retain_rate_list = [0.99,0.98,0.99,0.98,0.95,0.90,0.85,0.9,0.85]
retain_rate_list=[0.9,0.8,0.7]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 22.051M                | 3.555G      |
|  cls_token                 |  (1, 1, 384)           |             |
|  pos_embed                 |  (1, 197, 384)         |             |
|  patch_embed.proj          |  0.295M                |  57.803M    |
|   patch_embed.proj.weight  |   (384, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (384,)               |             |
|  blocks                    |  21.294M               |  3.496G     |
|   blocks.0                 |   1.774M               |   0.379G    |
|    blocks.0.norm1          |    0.768K              |    0.378M   |
|     blocks.0.norm1.weight  |     (384,)             |             |
|     blocks.0.norm1.bias    |     (384,)             |             |
|    blocks.0.attn           |    0.591M              |    0.146G   |
|     blocks.0.attn.

In [27]:
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 0 3 6 9 --retain_rate_list 0.6 0.42 0.5 0.5 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  1.980 ( 1.980)	Loss 1.8879e+00 (1.8879e+00)	Acc@1  67.97 ( 67.97)	Acc@5  81.25 ( 81.25)
Test: [ 20/391]	Time  0.958 ( 0.217)	Loss 1.7934e+00 (1.7024e+00)	Acc@1  67.19 ( 66.22)	Acc@5  80.47 ( 84.82)
Test: [ 40/391]	Time  0.934 ( 0.172)	Loss 2.4957e+00 (1.6417e+00)	Acc@1  50.78 ( 67.44)	Acc@5  71.09 ( 85.63)
Test: [ 60/391]	Time  0.803 ( 0.151)	Loss 1.7242e+00 (1.6112e+00)	Acc@1  63.28 ( 68.35)	Acc@5  82.81 ( 85.89)
Test: [ 80/391]	Time  0.771 ( 0.142)	Loss 2.0455e+00 (1.7491e+00)	Acc@1  53.91 ( 64.37)	Acc@5  83.59 ( 84.38)
Test: [100/391]	Time  0.834 ( 0.136)	

In [19]:
# after emphasizing
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 0 3 6 9 --retain_rate_list 0.6 0.42 0.5 0.5 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  1.929 ( 1.929)	Loss 1.2412e+00 (1.2412e+00)	Acc@1  78.91 ( 78.91)	Acc@5  92.19 ( 92.19)
Test: [ 20/391]	Time  0.891 ( 0.204)	Loss 1.9465e+00 (1.4719e+00)	Acc@1  64.06 ( 70.98)	Acc@5  77.34 ( 87.83)
Test: [ 40/391]	Time  0.849 ( 0.163)	Loss 2.0460e+00 (1.4491e+00)	Acc@1  61.72 ( 71.59)	Acc@5  78.12 ( 88.38)
Test: [ 60/391]	Time  0.766 ( 0.144)	Loss 1.7800e+00 (1.4472e+00)	Acc@1  64.06 ( 71.64)	Acc@5  81.25 ( 88.10)
Test: [ 80/391]	Time  0.769 ( 0.134)	Loss 2.1462e+00 (1.5937e+00)	Acc@1  50.78 ( 67.32)	Acc@5  76.56 ( 86.49)
Test: [100/391]	Time  0.777 ( 0.128)	

In [72]:
# average
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 0 3 6 9 --retain_rate_list 0.6 0.42 0.5 0.5 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  1.980 ( 1.980)	Loss 1.5944e+00 (1.5944e+00)	Acc@1  71.88 ( 71.88)	Acc@5  85.94 ( 85.94)
Test: [ 20/391]	Time  0.906 ( 0.213)	Loss 1.6819e+00 (1.5359e+00)	Acc@1  68.75 ( 68.68)	Acc@5  82.03 ( 86.90)
Test: [ 40/391]	Time  0.951 ( 0.171)	Loss 1.9657e+00 (1.4803e+00)	Acc@1  64.06 ( 69.87)	Acc@5  79.69 ( 87.92)
Test: [ 60/391]	Time  0.816 ( 0.151)	Loss 1.6340e+00 (1.4574e+00)	Acc@1  63.28 ( 70.77)	Acc@5  84.38 ( 87.95)
Test: [ 80/391]	Time  0.825 ( 0.141)	Loss 1.9996e+00 (1.5920e+00)	Acc@1  53.91 ( 66.80)	Acc@5  82.03 ( 86.68)
Test: [100/391]	Time  0.622 ( 0.134)	

In [34]:
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  1.988 ( 1.988)	Loss 4.4891e-01 (4.4891e-01)	Acc@1  91.41 ( 91.41)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.690 ( 0.206)	Loss 6.0852e-01 (6.6687e-01)	Acc@1  86.72 ( 85.45)	Acc@5  95.31 ( 97.06)
Test: [ 40/391]	Time  0.686 ( 0.165)	Loss 6.6657e-01 (6.9179e-01)	Acc@1  85.94 ( 84.74)	Acc@5  96.09 ( 96.93)
Test: [ 60/391]	Time  0.592 ( 0.145)	Loss 8.5541e-01 (6.5545e-01)	Acc@1  82.03 ( 86.23)	Acc@5  94.53 ( 96.99)
Test: [ 80/391]	Time  0.607 ( 0.135)	Loss 8.2408e-01 (7.1391e-01)	Acc@1  78.12 ( 84.51)	Acc@5  96.09 ( 96.76)
Test: [100/391]	Time  0.499 ( 0.128)	

In [1]:
# after emphasizing
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  2.242 ( 2.242)	Loss 4.7083e-01 (4.7083e-01)	Acc@1  91.41 ( 91.41)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.478 ( 0.500)	Loss 6.1043e-01 (6.5887e-01)	Acc@1  85.16 ( 85.86)	Acc@5  96.09 ( 97.36)
Test: [ 40/391]	Time  0.393 ( 0.433)	Loss 6.6422e-01 (6.8760e-01)	Acc@1  88.28 ( 84.89)	Acc@5  95.31 ( 97.08)
Test: [ 60/391]	Time  0.368 ( 0.415)	Loss 8.3477e-01 (6.5151e-01)	Acc@1  82.81 ( 86.40)	Acc@5  94.53 ( 97.14)
Test: [ 80/391]	Time  0.202 ( 0.405)	Loss 7.9118e-01 (7.0846e-01)	Acc@1  77.34 ( 84.69)	Acc@5  97.66 ( 96.88)
Test: [100/391]	Time  0.144 ( 0.362)	

In [2]:
# average
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  2.077 ( 2.077)	Loss 4.6547e-01 (4.6547e-01)	Acc@1  91.41 ( 91.41)	Acc@5  98.44 ( 98.44)
Test: [ 20/391]	Time  0.837 ( 0.228)	Loss 6.2792e-01 (6.6237e-01)	Acc@1  85.94 ( 85.83)	Acc@5  95.31 ( 97.17)
Test: [ 40/391]	Time  1.038 ( 0.189)	Loss 6.9242e-01 (6.9257e-01)	Acc@1  86.72 ( 84.87)	Acc@5  95.31 ( 97.01)
Test: [ 60/391]	Time  0.912 ( 0.172)	Loss 8.6182e-01 (6.5671e-01)	Acc@1  82.81 ( 86.21)	Acc@5  95.31 ( 97.04)
Test: [ 80/391]	Time  0.897 ( 0.163)	Loss 7.9644e-01 (7.1278e-01)	Acc@1  78.12 ( 84.53)	Acc@5  96.88 ( 96.80)
Test: [100/391]	Time  0.783 ( 0.156)	

In [17]:
!nvidia-smi

Tue May  3 03:07:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:21:00.0 Off |                    0 |
| N/A   36C    P0    39W / 250W |  38863MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# emphasizing after average
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  2.158 ( 2.158)	Loss 4.6547e-01 (4.6547e-01)	Acc@1  91.41 ( 91.41)	Acc@5  98.44 ( 98.44)
Test: [ 20/391]	Time  0.924 ( 0.240)	Loss 6.2792e-01 (6.6237e-01)	Acc@1  85.94 ( 85.83)	Acc@5  95.31 ( 97.17)
Test: [ 40/391]	Time  0.874 ( 0.192)	Loss 6.9242e-01 (6.9257e-01)	Acc@1  86.72 ( 84.87)	Acc@5  95.31 ( 97.01)
Test: [ 60/391]	Time  0.699 ( 0.167)	Loss 8.6182e-01 (6.5671e-01)	Acc@1  82.81 ( 86.21)	Acc@5  95.31 ( 97.04)
Test: [ 80/391]	Time  0.822 ( 0.157)	Loss 7.9644e-01 (7.1278e-01)	Acc@1  78.12 ( 84.53)	Acc@5  96.88 ( 96.80)
Test: [100/391]	Time  0.806 ( 0.151)	

In [4]:
# random
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  1.984 ( 1.984)	Loss 5.0387e-01 (5.0387e-01)	Acc@1  90.62 ( 90.62)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.845 ( 0.213)	Loss 6.8122e-01 (7.1524e-01)	Acc@1  88.28 ( 84.71)	Acc@5  95.31 ( 96.61)
Test: [ 40/391]	Time  0.866 ( 0.176)	Loss 6.6253e-01 (7.4280e-01)	Acc@1  87.50 ( 83.59)	Acc@5  94.53 ( 96.57)
Test: [ 60/391]	Time  0.810 ( 0.160)	Loss 8.1139e-01 (7.0165e-01)	Acc@1  81.25 ( 85.14)	Acc@5  96.88 ( 96.67)
Test: [ 80/391]	Time  0.802 ( 0.151)	Loss 8.3225e-01 (7.6430e-01)	Acc@1  78.12 ( 83.16)	Acc@5  96.88 ( 96.40)
Test: [100/391]	Time  0.804 ( 0.146)	

In [73]:
#no pruning
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 1 --retain_rate_list 1 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  2.161 ( 2.161)	Loss 4.3257e-01 (4.3257e-01)	Acc@1  91.41 ( 91.41)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.650 ( 0.223)	Loss 5.9604e-01 (6.4916e-01)	Acc@1  86.72 ( 85.71)	Acc@5  96.09 ( 97.28)
Test: [ 40/391]	Time  0.609 ( 0.176)	Loss 6.2325e-01 (6.7726e-01)	Acc@1  86.72 ( 84.78)	Acc@5  96.88 ( 97.10)
Test: [ 60/391]	Time  0.522 ( 0.156)	Loss 8.3152e-01 (6.4015e-01)	Acc@1  82.03 ( 86.28)	Acc@5  95.31 ( 97.13)
Test: [ 80/391]	Time  0.561 ( 0.145)	Loss 7.5286e-01 (6.9579e-01)	Acc@1  80.47 ( 84.69)	Acc@5  97.66 ( 96.90)
Test: [100/391]	Time  0.532 ( 0.138)	

In [62]:
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 2 3 4 5 6 7 8 9 10 --retain_rate_list 0.99 0.98 0.99 0.98 0.95 0.90 0.85 0.9 0.85 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  2.082 ( 2.082)	Loss 4.4167e-01 (4.4167e-01)	Acc@1  91.41 ( 91.41)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.598 ( 0.214)	Loss 5.9877e-01 (6.5262e-01)	Acc@1  86.72 ( 85.83)	Acc@5  95.31 ( 97.25)
Test: [ 40/391]	Time  0.551 ( 0.168)	Loss 6.4187e-01 (6.8102e-01)	Acc@1  89.06 ( 84.98)	Acc@5  96.09 ( 97.03)
Test: [ 60/391]	Time  0.480 ( 0.147)	Loss 8.4044e-01 (6.4450e-01)	Acc@1  82.03 ( 86.44)	Acc@5  95.31 ( 97.11)
Test: [ 80/391]	Time  0.510 ( 0.137)	Loss 7.6192e-01 (7.0062e-01)	Acc@1  80.47 ( 84.84)	Acc@5  96.88 ( 96.88)
Test: [100/391]	Time  0.489 ( 0.131)	

In [84]:
# after emphasizing
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_small_3' --prune_list 2 3 4 5 6 7 8 9 10 --retain_rate_list 0.99 0.98 0.99 0.98 0.95 0.90 0.85 0.9 0.85 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_small_patch16_224-cd65a155.pth
number of params: 22050664
Test: [  0/391]	Time  2.118 ( 2.118)	Loss 4.4143e-01 (4.4143e-01)	Acc@1  91.41 ( 91.41)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.620 ( 0.215)	Loss 5.9586e-01 (6.5236e-01)	Acc@1  86.72 ( 85.79)	Acc@5  95.31 ( 97.21)
Test: [ 40/391]	Time  0.587 ( 0.170)	Loss 6.4995e-01 (6.8075e-01)	Acc@1  88.28 ( 84.91)	Acc@5  95.31 ( 96.99)
Test: [ 60/391]	Time  0.544 ( 0.151)	Loss 8.3758e-01 (6.4397e-01)	Acc@1  82.03 ( 86.40)	Acc@5  95.31 ( 97.09)
Test: [ 80/391]	Time  0.508 ( 0.140)	Loss 7.5973e-01 (7.0003e-01)	Acc@1  80.47 ( 84.85)	Acc@5  96.88 ( 96.87)
Test: [100/391]	Time  0.534 ( 0.134)	

In [10]:
#Testing DEIT -S
patch_size = 16
layers = 12
prune_list = [2,5,8]
heads = 6
mlp_ratio = 4.
dims = 384
qkv_bias = True
retain_rate_list = [0.8, 0.8, 0.8]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 22.051M                | 3.344G      |
|  cls_token                 |  (1, 1, 384)           |             |
|  pos_embed                 |  (1, 197, 384)         |             |
|  patch_embed.proj          |  0.295M                |  57.803M    |
|   patch_embed.proj.weight  |   (384, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (384,)               |             |
|  blocks                    |  21.294M               |  3.286G     |
|   blocks.0                 |   1.774M               |   0.379G    |
|    blocks.0.norm1          |    0.768K              |    0.378M   |
|     blocks.0.norm1.weight  |     (384,)             |             |
|     blocks.0.norm1.bias    |     (384,)             |             |
|    blocks.0.attn           |    0.591M              |    0.146G   |
|     blocks.0.attn.

In [12]:
#Testing DEIT -S
patch_size = 16
layers = 12
prune_list = [2,5,8]
heads = 6
mlp_ratio = 4.
dims = 384
qkv_bias = True
retain_rate_list = [0.7,0.7,0.7]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 22.051M                | 2.873G      |
|  cls_token                 |  (1, 1, 384)           |             |
|  pos_embed                 |  (1, 197, 384)         |             |
|  patch_embed.proj          |  0.295M                |  57.803M    |
|   patch_embed.proj.weight  |   (384, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (384,)               |             |
|  blocks                    |  21.294M               |  2.815G     |
|   blocks.0                 |   1.774M               |   0.379G    |
|    blocks.0.norm1          |    0.768K              |    0.378M   |
|     blocks.0.norm1.weight  |     (384,)             |             |
|     blocks.0.norm1.bias    |     (384,)             |             |
|    blocks.0.attn           |    0.591M              |    0.146G   |
|     blocks.0.attn.

In [13]:
#Testing DEIT -S
patch_size = 16
layers = 12
prune_list = [2,5,8]
heads = 6
mlp_ratio = 4.
dims = 384
qkv_bias = True
retain_rate_list = [0.6,0.6,0.6]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 22.051M                | 2.478G      |
|  cls_token                 |  (1, 1, 384)           |             |
|  pos_embed                 |  (1, 197, 384)         |             |
|  patch_embed.proj          |  0.295M                |  57.803M    |
|   patch_embed.proj.weight  |   (384, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (384,)               |             |
|  blocks                    |  21.294M               |  2.42G      |
|   blocks.0                 |   1.774M               |   0.379G    |
|    blocks.0.norm1          |    0.768K              |    0.378M   |
|     blocks.0.norm1.weight  |     (384,)             |             |
|     blocks.0.norm1.bias    |     (384,)             |             |
|    blocks.0.attn           |    0.591M              |    0.146G   |
|     blocks.0.attn.

In [14]:
#Testing DEIT -B
patch_size = 16
layers = 12
prune_list = [2,5,8]
heads = 12
mlp_ratio = 4.
dims = 768
qkv_bias = True
retain_rate_list = [1,1,1]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops     |
|:---------------------------|:-----------------------|:-----------|
| model                      | 86.568M                | 17.583G    |
|  cls_token                 |  (1, 1, 768)           |            |
|  pos_embed                 |  (1, 197, 768)         |            |
|  patch_embed.proj          |  0.591M                |  0.116G    |
|   patch_embed.proj.weight  |   (768, 3, 16, 16)     |            |
|   patch_embed.proj.bias    |   (768,)               |            |
|  blocks                    |  85.054M               |  17.466G   |
|   blocks.0                 |   7.088M               |   1.455G   |
|    blocks.0.norm1          |    1.536K              |    0.756M  |
|     blocks.0.norm1.weight  |     (768,)             |            |
|     blocks.0.norm1.bias    |     (768,)             |            |
|    blocks.0.attn           |    2.362M              |    0.524G  |
|     blocks.0.attn.qkv      |    

In [76]:
#Testing DEIT -B
patch_size = 16
layers = 12
prune_list = [2,5,8]
heads = 12
mlp_ratio = 4.
dims = 768
qkv_bias = True
retain_rate_list = [0.9,0.9,0.9]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 86.568M                | 15.02G      |
|  cls_token                 |  (1, 1, 768)           |             |
|  pos_embed                 |  (1, 197, 768)         |             |
|  patch_embed.proj          |  0.591M                |  0.116G     |
|   patch_embed.proj.weight  |   (768, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (768,)               |             |
|  blocks                    |  85.054M               |  14.903G    |
|   blocks.0                 |   7.088M               |   1.455G    |
|    blocks.0.norm1          |    1.536K              |    0.756M   |
|     blocks.0.norm1.weight  |     (768,)             |             |
|     blocks.0.norm1.bias    |     (768,)             |             |
|    blocks.0.attn           |    2.362M              |    0.524G   |
|     blocks.0.attn.

In [78]:
#Testing DEIT -B
patch_size = 16
layers = 12
prune_list = [2,5,8]
heads = 12
mlp_ratio = 4.
dims = 768
qkv_bias = True
retain_rate_list = [0.9,0.8,0.7]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 86.568M                | 13.628G     |
|  cls_token                 |  (1, 1, 768)           |             |
|  pos_embed                 |  (1, 197, 768)         |             |
|  patch_embed.proj          |  0.591M                |  0.116G     |
|   patch_embed.proj.weight  |   (768, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (768,)               |             |
|  blocks                    |  85.054M               |  13.512G    |
|   blocks.0                 |   7.088M               |   1.455G    |
|    blocks.0.norm1          |    1.536K              |    0.756M   |
|     blocks.0.norm1.weight  |     (768,)             |             |
|     blocks.0.norm1.bias    |     (768,)             |             |
|    blocks.0.attn           |    2.362M              |    0.524G   |
|     blocks.0.attn.

In [79]:
# no pruning
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_base_3' --prune_list 2 --retain_rate_list 1 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_base_patch16_224-b5f2ef4d.pth
number of params: 86567656
Test: [  0/391]	Time  2.257 ( 2.257)	Loss 2.9628e-01 (2.9628e-01)	Acc@1  96.09 ( 96.09)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.123 ( 0.235)	Loss 4.9533e-01 (5.8144e-01)	Acc@1  87.50 ( 87.31)	Acc@5  98.44 ( 97.69)
Test: [ 40/391]	Time  0.132 ( 0.184)	Loss 5.4839e-01 (6.2589e-01)	Acc@1  92.97 ( 86.34)	Acc@5  94.53 ( 97.31)
Test: [ 60/391]	Time  0.130 ( 0.167)	Loss 8.0301e-01 (5.8726e-01)	Acc@1  84.38 ( 87.72)	Acc@5  95.31 ( 97.59)
Test: [ 80/391]	Time  0.130 ( 0.158)	Loss 6.2037e-01 (6.2592e-01)	Acc@1  85.94 ( 86.60)	Acc@5  97.66 ( 97.47)
Test: [100/391]	Time  0.143 ( 0.153)	L

In [77]:
# average
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_base_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_base_patch16_224-b5f2ef4d.pth
number of params: 86567656
Test: [  0/391]	Time  2.094 ( 2.094)	Loss 2.9283e-01 (2.9283e-01)	Acc@1  96.09 ( 96.09)	Acc@5 100.00 (100.00)
Test: [ 20/391]	Time  0.201 ( 0.216)	Loss 5.0721e-01 (5.9673e-01)	Acc@1  89.06 ( 87.28)	Acc@5  98.44 ( 97.81)
Test: [ 40/391]	Time  0.173 ( 0.172)	Loss 5.9670e-01 (6.4031e-01)	Acc@1  91.41 ( 86.20)	Acc@5  93.75 ( 97.37)
Test: [ 60/391]	Time  0.101 ( 0.152)	Loss 7.8141e-01 (5.9826e-01)	Acc@1  83.59 ( 87.69)	Acc@5  96.09 ( 97.61)
Test: [ 80/391]	Time  0.108 ( 0.141)	Loss 6.6434e-01 (6.3982e-01)	Acc@1  84.38 ( 86.44)	Acc@5  98.44 ( 97.48)
Test: [100/391]	Time  0.100 ( 0.134)	L

In [80]:
# after emphasizing
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_base_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_base_patch16_224-b5f2ef4d.pth
number of params: 86567656
Test: [  0/391]	Time  2.319 ( 2.319)	Loss 2.9164e-01 (2.9164e-01)	Acc@1  96.09 ( 96.09)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.293 ( 0.235)	Loss 4.7100e-01 (5.9716e-01)	Acc@1  89.84 ( 86.98)	Acc@5  99.22 ( 97.73)
Test: [ 40/391]	Time  0.184 ( 0.184)	Loss 5.9715e-01 (6.3801e-01)	Acc@1  90.62 ( 86.11)	Acc@5  93.75 ( 97.35)
Test: [ 60/391]	Time  0.131 ( 0.161)	Loss 8.0250e-01 (5.9460e-01)	Acc@1  82.81 ( 87.60)	Acc@5  95.31 ( 97.57)
Test: [ 80/391]	Time  0.189 ( 0.150)	Loss 6.4538e-01 (6.3503e-01)	Acc@1  83.59 ( 86.38)	Acc@5  98.44 ( 97.42)
Test: [100/391]	Time  0.113 ( 0.143)	L

In [81]:
# average after emphasizing
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_base_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_base_patch16_224-b5f2ef4d.pth
number of params: 86567656
Test: [  0/391]	Time  2.233 ( 2.233)	Loss 2.9018e-01 (2.9018e-01)	Acc@1  96.09 ( 96.09)	Acc@5  99.22 ( 99.22)
Test: [ 20/391]	Time  0.193 ( 0.217)	Loss 4.7988e-01 (5.9896e-01)	Acc@1  88.28 ( 86.98)	Acc@5  99.22 ( 97.81)
Test: [ 40/391]	Time  0.199 ( 0.170)	Loss 6.2485e-01 (6.4062e-01)	Acc@1  90.62 ( 86.13)	Acc@5  94.53 ( 97.43)
Test: [ 60/391]	Time  0.115 ( 0.151)	Loss 7.8889e-01 (6.0023e-01)	Acc@1  84.38 ( 87.60)	Acc@5  95.31 ( 97.62)
Test: [ 80/391]	Time  0.134 ( 0.141)	Loss 6.7546e-01 (6.4210e-01)	Acc@1  82.81 ( 86.23)	Acc@5  98.44 ( 97.49)
Test: [100/391]	Time  0.126 ( 0.135)	L

In [4]:
# random
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_base_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_base_patch16_224-b5f2ef4d.pth
number of params: 86567656
Traceback (most recent call last):
  File "/scratch/gpfs/hw4948/tokenrank_pruning/notebooks/../zero_shot_pruning_eval.py", line 259, in <module>
    main(args)
  File "/scratch/gpfs/hw4948/tokenrank_pruning/notebooks/../zero_shot_pruning_eval.py", line 157, in main
    validate(data_loader_val, model, criterion)
  File "/scratch/gpfs/hw4948/tokenrank_pruning/notebooks/../zero_shot_pruning_eval.py", line 234, in validate
    output = model(images)
  File "/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return f

In [6]:
# after emphasizing
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_base_3' --prune_list 0 3 6 9 --retain_rate_list 0.6 0.42 0.5 0.5 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_base_patch16_224-b5f2ef4d.pth
number of params: 86567656
Traceback (most recent call last):
  File "/scratch/gpfs/hw4948/tokenrank_pruning/notebooks/../zero_shot_pruning_eval.py", line 259, in <module>
    main(args)
  File "/scratch/gpfs/hw4948/tokenrank_pruning/notebooks/../zero_shot_pruning_eval.py", line 157, in main
    validate(data_loader_val, model, criterion)
  File "/scratch/gpfs/hw4948/tokenrank_pruning/notebooks/../zero_shot_pruning_eval.py", line 234, in validate
    output = model(images)
  File "/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return f

In [16]:
#Testing DEIT -T
patch_size = 16
layers = 12
prune_list = [2,5,8]
heads = 3
mlp_ratio = 4.
dims = 192
qkv_bias = True
retain_rate_list = [1,1,1]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 5.717M                 | 1.259G      |
|  cls_token                 |  (1, 1, 192)           |             |
|  pos_embed                 |  (1, 197, 192)         |             |
|  patch_embed.proj          |  0.148M                |  28.901M    |
|   patch_embed.proj.weight  |   (192, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (192,)               |             |
|  blocks                    |  5.338M                |  1.23G      |
|   blocks.0                 |   0.445M               |   0.102G    |
|    blocks.0.norm1          |    0.384K              |    0.189M   |
|     blocks.0.norm1.weight  |     (192,)             |             |
|     blocks.0.norm1.bias    |     (192,)             |             |
|    blocks.0.attn           |    0.148M              |    43.951M  |
|     blocks.0.attn.

In [11]:
#Testing DEIT -T
patch_size = 16
layers = 12
prune_list = [2,5,8]
heads = 3
mlp_ratio = 4.
dims = 192
qkv_bias = True
retain_rate_list = [0.9,0.8,0.7]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 5.717M                 | 0.963G      |
|  cls_token                 |  (1, 1, 192)           |             |
|  pos_embed                 |  (1, 197, 192)         |             |
|  patch_embed.proj          |  0.148M                |  28.901M    |
|   patch_embed.proj.weight  |   (192, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (192,)               |             |
|  blocks                    |  5.338M                |  0.933G     |
|   blocks.0                 |   0.445M               |   0.102G    |
|    blocks.0.norm1          |    0.384K              |    0.189M   |
|     blocks.0.norm1.weight  |     (192,)             |             |
|     blocks.0.norm1.bias    |     (192,)             |             |
|    blocks.0.attn           |    0.148M              |    43.951M  |
|     blocks.0.attn.

In [13]:
#Testing DEIT -T
patch_size = 16
layers = 12
prune_list = [2,5,8,9,10,11]
heads = 3
mlp_ratio = 4.
dims = 192
qkv_bias = True
retain_rate_list = [0.8,0.8,0.7,0.7,0.7,0.7]
tau_imp = 1
input_size=224
model = TokenRankVisionTransformer(
                prune_list=prune_list, patch_size=patch_size,  embed_dim=dims, depth=layers,
                num_heads=heads, mlp_ratio=mlp_ratio,qkv_bias=qkv_bias,retain_rate_list = retain_rate_list,
                tau_imp=tau_imp)
model = model.eval()
device = 'cuda:0'
model = model.to(device)
inputs = inputs.to(device)
# op = model(inputs)
flop = FlopCountAnalysis(model, inputs)
print(flop_count_table(flop, max_depth=4))
print(flop_count_str(flop))
print(flop.total())

| module                     | #parameters or shape   | #flops      |
|:---------------------------|:-----------------------|:------------|
| model                      | 5.717M                 | 0.849G      |
|  cls_token                 |  (1, 1, 192)           |             |
|  pos_embed                 |  (1, 197, 192)         |             |
|  patch_embed.proj          |  0.148M                |  28.901M    |
|   patch_embed.proj.weight  |   (192, 3, 16, 16)     |             |
|   patch_embed.proj.bias    |   (192,)               |             |
|  blocks                    |  5.338M                |  0.819G     |
|   blocks.0                 |   0.445M               |   0.102G    |
|    blocks.0.norm1          |    0.384K              |    0.189M   |
|     blocks.0.norm1.weight  |     (192,)             |             |
|     blocks.0.norm1.bias    |     (192,)             |             |
|    blocks.0.attn           |    0.148M              |    43.951M  |
|     blocks.0.attn.

In [3]:
# empharsizing after average
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_tiny_3' --prune_list 2 5 8 9 10 11 --retain_rate_list 0.9 0.8 0.9 0.9 0.9 0.9 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_tiny_patch16_224-a1311bcf.pth
number of params: 5717416
Test: [  0/391]	Time  2.094 ( 2.094)	Loss 6.5117e-01 (6.5117e-01)	Acc@1  85.94 ( 85.94)	Acc@5  96.09 ( 96.09)
Test: [ 20/391]	Time  0.901 ( 0.216)	Loss 1.0334e+00 (9.2292e-01)	Acc@1  78.91 ( 80.02)	Acc@5  93.75 ( 94.49)
Test: [ 40/391]	Time  0.933 ( 0.171)	Loss 1.0122e+00 (9.4529e-01)	Acc@1  79.69 ( 79.31)	Acc@5  92.19 ( 94.26)
Test: [ 60/391]	Time  0.856 ( 0.152)	Loss 1.0657e+00 (9.1658e-01)	Acc@1  78.12 ( 80.65)	Acc@5  92.97 ( 94.38)
Test: [ 80/391]	Time  0.851 ( 0.142)	Loss 1.1886e+00 (9.9081e-01)	Acc@1  72.66 ( 77.98)	Acc@5  90.62 ( 94.09)
Test: [100/391]	Time  0.793 ( 0.136)	Lo

In [5]:
# random
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_tiny_3' --prune_list 2 5 8 9 10 11 --retain_rate_list 0.9 0.8 0.9 0.9 0.9 0.9 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_tiny_patch16_224-a1311bcf.pth
number of params: 5717416
Test: [  0/391]	Time  2.054 ( 2.054)	Loss 7.1200e-01 (7.1200e-01)	Acc@1  86.72 ( 86.72)	Acc@5  95.31 ( 95.31)
Test: [ 20/391]	Time  1.022 ( 0.229)	Loss 1.0543e+00 (9.7437e-01)	Acc@1  81.25 ( 79.46)	Acc@5  92.97 ( 93.71)
Test: [ 40/391]	Time  1.089 ( 0.185)	Loss 1.0446e+00 (1.0014e+00)	Acc@1  78.91 ( 78.39)	Acc@5  89.84 ( 93.46)
Test: [ 60/391]	Time  1.008 ( 0.167)	Loss 1.1876e+00 (9.8588e-01)	Acc@1  77.34 ( 79.18)	Acc@5  92.19 ( 93.62)
Test: [ 80/391]	Time  1.031 ( 0.157)	Loss 1.2243e+00 (1.0636e+00)	Acc@1  71.88 ( 76.34)	Acc@5  92.19 ( 93.40)
Test: [100/391]	Time  0.912 ( 0.151)	Lo

In [1]:
# empharsizing after average
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_tiny_3' --prune_list 2 5 8 9 10 11 --retain_rate_list 0.8 0.8 0.7 0.7 0.7 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_tiny_patch16_224-a1311bcf.pth
number of params: 5717416
Test: [  0/391]	Time  2.190 ( 2.190)	Loss 7.2974e-01 (7.2974e-01)	Acc@1  86.72 ( 86.72)	Acc@5  95.31 ( 95.31)
Test: [ 20/391]	Time  1.141 ( 0.245)	Loss 1.0566e+00 (9.8417e-01)	Acc@1  78.12 ( 78.79)	Acc@5  94.53 ( 94.20)
Test: [ 40/391]	Time  1.048 ( 0.192)	Loss 1.1588e+00 (1.0091e+00)	Acc@1  79.69 ( 78.24)	Acc@5  90.62 ( 93.75)
Test: [ 60/391]	Time  0.990 ( 0.170)	Loss 1.1293e+00 (9.8887e-01)	Acc@1  76.56 ( 79.12)	Acc@5  92.97 ( 93.90)
Test: [ 80/391]	Time  1.051 ( 0.159)	Loss 1.2499e+00 (1.0673e+00)	Acc@1  70.31 ( 76.44)	Acc@5  90.62 ( 93.60)
Test: [100/391]	Time  1.049 ( 0.153)	Lo

In [4]:
# random
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_tiny_3' --prune_list 2 5 8 9 10 11 --retain_rate_list 0.8 0.8 0.7 0.7 0.7 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_tiny_patch16_224-a1311bcf.pth
number of params: 5717416
Test: [  0/391]	Time  2.102 ( 2.102)	Loss 1.0357e+00 (1.0357e+00)	Acc@1  81.25 ( 81.25)	Acc@5  95.31 ( 95.31)
Test: [ 20/391]	Time  1.055 ( 0.233)	Loss 1.3838e+00 (1.2149e+00)	Acc@1  77.34 ( 75.78)	Acc@5  89.06 ( 92.56)
Test: [ 40/391]	Time  1.042 ( 0.184)	Loss 1.3204e+00 (1.2433e+00)	Acc@1  74.22 ( 75.15)	Acc@5  89.84 ( 92.09)
Test: [ 60/391]	Time  0.865 ( 0.161)	Loss 1.3326e+00 (1.2276e+00)	Acc@1  69.53 ( 75.70)	Acc@5  92.19 ( 92.26)
Test: [ 80/391]	Time  0.951 ( 0.151)	Loss 1.5748e+00 (1.3123e+00)	Acc@1  64.06 ( 72.44)	Acc@5  89.84 ( 91.74)
Test: [100/391]	Time  0.810 ( 0.145)	Lo

In [5]:
# average
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_tiny_3' --prune_list 2 5 8 9 10 11 --retain_rate_list 0.9 0.8 0.9 0.9 0.9 0.9 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_tiny_patch16_224-a1311bcf.pth
number of params: 5717416
Test: [  0/391]	Time  2.172 ( 2.172)	Loss 6.3284e-01 (6.3284e-01)	Acc@1  87.50 ( 87.50)	Acc@5  96.09 ( 96.09)
Test: [ 20/391]	Time  1.127 ( 0.245)	Loss 1.0295e+00 (9.1775e-01)	Acc@1  78.91 ( 80.02)	Acc@5  92.97 ( 94.49)
Test: [ 40/391]	Time  1.047 ( 0.192)	Loss 1.0082e+00 (9.4325e-01)	Acc@1  79.69 ( 79.44)	Acc@5  92.19 ( 94.13)
Test: [ 60/391]	Time  0.936 ( 0.169)	Loss 1.0758e+00 (9.1470e-01)	Acc@1  78.12 ( 80.57)	Acc@5  92.19 ( 94.25)
Test: [ 80/391]	Time  1.082 ( 0.161)	Loss 1.2096e+00 (9.9096e-01)	Acc@1  71.88 ( 77.85)	Acc@5  90.62 ( 94.06)
Test: [100/391]	Time  1.154 ( 0.157)	Lo

In [6]:
# no pruning
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_tiny_3' --prune_list 2 --retain_rate_list 1 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_tiny_patch16_224-a1311bcf.pth
number of params: 5717416
Test: [  0/391]	Time  2.093 ( 2.093)	Loss 5.9033e-01 (5.9033e-01)	Acc@1  88.28 ( 88.28)	Acc@5  97.66 ( 97.66)
Test: [ 20/391]	Time  0.964 ( 0.230)	Loss 9.9164e-01 (8.7250e-01)	Acc@1  80.47 ( 81.32)	Acc@5  93.75 ( 95.39)
Test: [ 40/391]	Time  0.974 ( 0.183)	Loss 9.1898e-01 (8.9883e-01)	Acc@1  80.47 ( 80.34)	Acc@5  91.41 ( 94.84)
Test: [ 60/391]	Time  0.852 ( 0.162)	Loss 1.0682e+00 (8.7181e-01)	Acc@1  78.91 ( 81.19)	Acc@5  93.75 ( 94.95)
Test: [ 80/391]	Time  0.837 ( 0.150)	Loss 1.1435e+00 (9.4586e-01)	Acc@1  71.88 ( 78.51)	Acc@5  91.41 ( 94.75)
Test: [100/391]	Time  0.834 ( 0.142)	Lo

In [4]:
# empharsizing after average
!python3 ../zero_shot_pruning_eval.py --model_name='tokenrank_deit_tiny_3' --prune_list 2 5 8 --retain_rate_list 0.9 0.8 0.7 --data-path='/scratch/gpfs/hw4948/ILSVRC2012'

/home/hw4948/.conda/envs/tokenrank/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
# missing keys= []
# unexpected keys= []
successfully loaded from pre-trained weights: ../model_weights/deit_tiny_patch16_224-a1311bcf.pth
number of params: 5717416
Test: [  0/391]	Time  1.906 ( 1.906)	Loss 6.6532e-01 (6.6532e-01)	Acc@1  85.16 ( 85.16)	Acc@5  96.09 ( 96.09)
Test: [ 20/391]	Time  0.904 ( 0.213)	Loss 1.0347e+00 (9.2631e-01)	Acc@1  79.69 ( 79.84)	Acc@5  92.97 ( 94.46)
Test: [ 40/391]	Time  0.987 ( 0.172)	Loss 1.0321e+00 (9.4774e-01)	Acc@1  78.91 ( 79.17)	Acc@5  91.41 ( 94.21)
Test: [ 60/391]	Time  0.867 ( 0.152)	Loss 1.0741e+00 (9.2113e-01)	Acc@1  77.34 ( 80.35)	Acc@5  92.97 ( 94.33)
Test: [ 80/391]	Time  0.896 ( 0.143)	Loss 1.1870e+00 (9.9649e-01)	Acc@1  72.66 ( 77.75)	Acc@5  90.62 ( 94.04)
Test: [100/391]	Time  0.807 ( 0.137)	Lo